# Produce the results and figures

AGU's figure guidelines state:

Figure sizes should be between 95 x 115 mm for quarter page to 190 x 230 mm for full page. https://www.agu.org/Publish-with-AGU/Publish/Author-Resources/Graphic-Requirements

## Import required modules

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as pl
import glob
from scipy.io.idl import readsav
from scipy.stats import linregress
from matplotlib.patches import Rectangle

## Set up plotting defaults

In [ ]:
pl.rcParams['font.size'] = 9
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.top'] = True

In [ ]:
colors = {
    'cmip5': '#cc2323',
    'cmip6': '#2551cc',
}

## Get pre-calculated models and pre-industrial control branch points

In [ ]:
with open('../data_output/branch_points.json', 'r') as f:
    branch_points = json.load(f)

In [ ]:
models = list(branch_points['historical'].keys())
models

In [ ]:
piControls = {
    'ACCESS-CM2': 'r1i1p1f1',
    'ACCESS-ESM1-5': 'r1i1p1f1',
    'AWI-CM-1-1-MR': 'r1i1p1f1',
    'CAMS-CSM1-0': 'r1i1p1f1',
    'CanESM5': 'r1i1p1f1',
    'CESM2': 'r1i1p1f1',
    'CESM2-FV2': 'r1i1p1f1',
    'CESM2-WACCM': 'r1i1p1f1',
    'CESM2-WACCM-FV2': 'r1i1p1f1',
    'CIESM': 'r1i1p1f1',
    'CMCC-CM2-SR5': 'r1i1p1f1',
    'CMCC-ESM2': 'r1i1p1f1',
    'CNRM-CM6-1': 'r1i1p1f2',
    'CNRM-ESM2-1': 'r1i1p1f2',
    'E3SM-1-0': 'r1i1p1f1',
    'EC-Earth3': 'r1i1p1f1',
    'EC-Earth3-AerChem': 'r1i1p1f1',
    'FGOALS-g3': 'r1i1p1f1',
    'GFDL-CM4': 'r1i1p1f1',
    'GFDL-ESM4': 'r1i1p1f1',
    'GISS-E2-1-G': 'r1i1p1f2',
    'HadGEM3-GC31-LL': 'r1i1p1f1',
    'HadGEM3-GC31-MM': 'r1i1p1f1',
    'INM-CM4-8': 'r1i1p1f1',
    'INM-CM5-0': 'r1i1p1f1',
    'IPSL-CM5A2-INCA': 'r1i1p1f1',
    'IPSL-CM6A-LR': 'r1i1p1f1',
    'KACE-1-0-G': 'r1i1p1f1',
    'MIROC6': 'r1i1p1f1',
    'MIROC-ES2L': 'r1i1p1f2',
    'MPI-ESM-1-2-HAM': 'r1i1p1f1',
    'MPI-ESM1-2-HR': 'r1i1p1f1',
    'MRI-ESM2-0': 'r1i1p1f1',
    'NESM3': 'r1i1p1f1',
    'NorESM2-LM': 'r1i1p1f1',
    'NorESM2-MM': 'r1i1p1f1',
    'TaiESM1': 'r1i1p1f1',
    'UKESM1-0-LL': 'r1i1p1f2',
}

In [ ]:
with open('../data_input/cmip56_forcing_feedback_ecs.json', 'rb') as f:
    feedbacks = json.load(f)

## Get pre-calculated climate feedback from Mark Zelinka

In [ ]:
lambda5 = {}
for model in feedbacks['CMIP5']:
    lambda5[model] = feedbacks['CMIP5'][model]['r1i1p1']['NET']
    
lambda6 = {}
for model in feedbacks['CMIP6']:
    if model not in piControls:
        continue
    fourtimes_run = piControls[model]
    if model=='EC-Earth3':  # no, I don't know why either
        fourtimes_run = 'r8i1p1f1'
    elif model=='GISS-E2-1-G':
        fourtimes_run = 'r1i1p1f1'
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        fourtimes_run = 'r1i1p1f3'
    lambda6[model] = feedbacks['CMIP6'][model][fourtimes_run]['NET']

## Calculate the implied ERF

In [ ]:
piControl = {}
experiments = {}

In [ ]:
for model in models:
    piControl[model] = pd.read_csv('../data_output/cmip6/%s/%s/piControl.csv' % (model, piControls[model]))
piControl['CanESM5-cmip5'] = pd.read_csv('../data_output/cmip6/CanESM5/r1i1p1f1/piControl-cmip5.csv')

In [ ]:
delta_N = {}
delta_T = {}
delta_F = {}

for experiment in ['historical', 'hist-GHG', 'hist-nat', 'hist-aer', 'historical-cmip5', 'hist-GHG-cmip5', 'hist-nat-cmip5', 'hist-aer-cmip5']:
    if experiment[-5:] == 'cmip5':
        nyears = 156
    else:
        nyears = 165
    delta_N[experiment] = {}
    delta_T[experiment] = {}
    delta_F[experiment] = {}
    for model in branch_points[experiment].keys():
        delta_N[experiment][model] = {}
        delta_T[experiment][model] = {}
        delta_F[experiment][model] = {}
        for run in list(branch_points[experiment][model].keys()):
            data = pd.read_csv('../data_output/cmip6/%s/%s/%s.csv' % (model, run, experiment))
            index_start = branch_points[experiment][model][run]
            N_historical = data['rsdt'].values[:nyears] - data['rsut'].values[:nyears] - data['rlut'].values[:nyears]
            if experiment[-5:] == 'cmip5':
                piC = piControl['CanESM5-cmip5']
            else:
                piC = piControl[model]
            N_piControl = (
                piC['rsdt'][index_start:index_start+nyears].values - 
                piC['rsut'][index_start:index_start+nyears].values - 
                piC['rlut'][index_start:index_start+nyears].values
            )
            delta_N[experiment][model][run] = N_historical - N_piControl
            delta_T[experiment][model][run] = data['tas'].values[:nyears] - piC['tas'][index_start:index_start+nyears].values
            delta_F[experiment][model][run] = delta_N[experiment][model][run] - lambda6[model] * delta_T[experiment][model][run]
        delta_F_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        delta_N_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        delta_T_array = np.ones((nyears, len(branch_points[experiment][model]))) * np.nan
        for i, run in enumerate(branch_points[experiment][model].keys()):
            delta_F_array[:, i] = delta_F[experiment][model][run]
            delta_N_array[:, i] = delta_N[experiment][model][run]
            delta_T_array[:, i] = delta_T[experiment][model][run]
        delta_F[experiment][model]['mean'] = np.mean(delta_F_array, axis=1)
        delta_N[experiment][model]['mean'] = np.mean(delta_N_array, axis=1)
        delta_T[experiment][model]['mean'] = np.mean(delta_T_array, axis=1)
        
        
delta_N['hist-otheranthro'] = {}
delta_T['hist-otheranthro'] = {}
delta_F['hist-otheranthro'] = {}
for model in branch_points['historical'].keys():
    if (model not in branch_points['hist-nat'].keys()) and (model not in branch_points['hist-GHG'].keys()):
        continue
    delta_N['hist-otheranthro'][model] = {}
    delta_T['hist-otheranthro'][model] = {}
    delta_F['hist-otheranthro'][model] = {}
    for run in list(branch_points['historical'][model].keys()):
        if (run not in branch_points['hist-nat'][model].keys()) or (run not in branch_points['hist-GHG'][model].keys()):
            continue
        delta_N['hist-otheranthro'][model][run] = delta_N['historical'][model][run] - delta_N['hist-GHG'][model][run] - delta_N['hist-nat'][model][run]
        delta_T['hist-otheranthro'][model][run] = delta_T['historical'][model][run] - delta_T['hist-GHG'][model][run] - delta_T['hist-nat'][model][run]
        delta_F['hist-otheranthro'][model][run] = delta_F['historical'][model][run] - delta_F['hist-GHG'][model][run] - delta_F['hist-nat'][model][run]
    delta_F_array = np.ones((165, len(delta_F['hist-otheranthro'][model]))) * np.nan
    delta_N_array = np.ones((165, len(delta_N['hist-otheranthro'][model]))) * np.nan
    delta_T_array = np.ones((165, len(delta_T['hist-otheranthro'][model]))) * np.nan
    for i, run in enumerate(delta_T['hist-otheranthro'][model].keys()):
        delta_F_array[:, i] = delta_F['hist-otheranthro'][model][run]
        delta_N_array[:, i] = delta_N['hist-otheranthro'][model][run]
        delta_T_array[:, i] = delta_T['hist-otheranthro'][model][run]
    delta_F['hist-otheranthro'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N['hist-otheranthro'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T['hist-otheranthro'][model]['mean'] = np.mean(delta_T_array, axis=1)

delta_N['hist-otheranthro-cmip5'] = {}
delta_T['hist-otheranthro-cmip5'] = {}
delta_F['hist-otheranthro-cmip5'] = {}
for model in branch_points['historical-cmip5'].keys():
    if (model not in branch_points['hist-nat-cmip5'].keys()) and (model not in branch_points['hist-GHG-cmip5'].keys()):
        continue
    delta_N['hist-otheranthro-cmip5'][model] = {}
    delta_T['hist-otheranthro-cmip5'][model] = {}
    delta_F['hist-otheranthro-cmip5'][model] = {}
    for run in list(branch_points['historical-cmip5'][model].keys()):
        if (run not in branch_points['hist-nat-cmip5'][model].keys()) or (run not in branch_points['hist-GHG-cmip5'][model].keys()):
            continue
        delta_N['hist-otheranthro-cmip5'][model][run] = delta_N['historical-cmip5'][model][run] - delta_N['hist-GHG-cmip5'][model][run] - delta_N['hist-nat-cmip5'][model][run]
        delta_T['hist-otheranthro-cmip5'][model][run] = delta_T['historical-cmip5'][model][run] - delta_T['hist-GHG-cmip5'][model][run] - delta_T['hist-nat-cmip5'][model][run]
        delta_F['hist-otheranthro-cmip5'][model][run] = delta_F['historical-cmip5'][model][run] - delta_F['hist-GHG-cmip5'][model][run] - delta_F['hist-nat-cmip5'][model][run]
    delta_F_array = np.ones((156, len(delta_F['hist-otheranthro-cmip5'][model]))) * np.nan
    delta_N_array = np.ones((156, len(delta_N['hist-otheranthro-cmip5'][model]))) * np.nan
    delta_T_array = np.ones((156, len(delta_T['hist-otheranthro-cmip5'][model]))) * np.nan
    for i, run in enumerate(delta_T['hist-otheranthro-cmip5'][model].keys()):
        delta_F_array[:, i] = delta_F['hist-otheranthro-cmip5'][model][run]
        delta_N_array[:, i] = delta_N['hist-otheranthro-cmip5'][model][run]
        delta_T_array[:, i] = delta_T['hist-otheranthro-cmip5'][model][run]
    delta_F['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T['hist-otheranthro-cmip5'][model]['mean'] = np.mean(delta_T_array, axis=1)

## Grab pre-calculated RFMIP-ERF forcing

In [ ]:
# grab the forcing, which was produced by me from RFMIP models in another project!
rfmip_erf = pd.read_csv('../data_input/RFMIP-ERF-tier2.csv')

## Plot Figure S1

Correlation between implied ERF and RFMIP ERF

Reviewer 2 wanted to see this. All models will be highly correlated - it might be best to look at the slopes and those models that are closer to 1:1 are better

In [ ]:
# Hand order models by slope
models_rfmip = ['GISS-E2-1-G', 'MIROC6', 'CNRM-CM6-1', 'CanESM5', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'GFDL-CM4', 'NorESM2-LM']

fig, ax = pl.subplots(4, 2, figsize=(19/2.54, 23/2.54))
for i, model in enumerate(models_rfmip):
    col = i%2
    row = i//2
    if '%s TOT' % model in rfmip_erf:
        ax[row,col].scatter(rfmip_erf['%s TOT' % model].values[:165], delta_F['historical'][model]['mean'], color=colors['cmip6'])
    ax[row,col].set_title('(%s) %s' % (chr(97+i), model))
    ax[row,col].set_xlabel('RFMIP Fixed-SST ERF (W m$^{-2}$)')
    ax[row,col].set_ylabel('implied ERF (W m$^{-2}$)')
    ax[row,col].grid()
    regress = linregress(rfmip_erf['%s TOT' % model].values[:165], delta_F['historical'][model]['mean'])
    print(model, regress.slope, regress.rvalue)
    x_min = np.min(rfmip_erf['%s TOT' % model].values[:165])
    x_max = np.max(rfmip_erf['%s TOT' % model].values[:165])
    y_min = np.min(delta_F['historical'][model]['mean'])
    y_max = np.max(delta_F['historical'][model]['mean'])
    
    # plot regression slope
    ax[row,col].plot(np.linspace(x_min-0.2, x_max+0.2), regress.slope*np.linspace(x_min-0.2, x_max+0.2)+regress.intercept, color='k', label='Best fit')
    
    # plot 1:1 line
    ax[row,col].plot(np.linspace(x_min-0.2, x_max+0.2), np.linspace(x_min-0.2, x_max+0.2), color='0.4', ls='--', label='1:1 line')
    
    # axis bounds
    #ax[row,col].set_xlim(x_min-0.2, x_max+0.2)
    #ax[row,col].set_ylim(y_min-0.2, y_max+0.2)
    ax[row,col].set_xlim(-2.3, 3)
    ax[row,col].set_ylim(-2.2, 2.4)
    
    ax[row,col].legend()
    ax[row,col].text(0.95,0.05,'slope = %.2f' % regress.slope, ha='right', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
    ax[row,col].text(0.95,0.17,'r = %.2f' % regress.rvalue, ha='right', va='bottom', transform=ax[row,col].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.1',ec='w',fc='w'))
    
#pl.figtext(0.01, 0.5, 'Historical effective radiative forcing (W m$^{-2}$)', rotation=90, va='center')
#ax[0,0].legend(loc='upper left', bbox_to_anchor=[0, -2.4], ncol=3)
fig.tight_layout(rect=[0.02, 0, 1, 1])
pl.savefig('../plots/figS1.pdf')

## Plot fig. 2

In [ ]:
fig, ax = pl.subplots(4, 2, figsize=(19/2.54, 23/2.54))
for i, model in enumerate(models_rfmip):
    col = i%2
    row = i//2
    for run in branch_points['historical'][model].keys():
        if model=='CanESM5' and run=='r1i1p1f1':
            label='implied ERF ensemble members'
        else:
            label=''
        ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model][run], color='0.7', label=label)
    ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model]['mean'], color='k', label='implied ERF ensemble mean')
    ax[row,col].set_title('(%s) %s' % (chr(97+i), model))
    if '%s TOT' % model in rfmip_erf:
        ax[row,col].plot(np.arange(1850.5, 2015), rfmip_erf['%s TOT' % model].values[:165], color=colors['cmip6'], label='RFMIP ERF')
    ax[row,col].set_xlim(1850, 2015)
    ax[row,col].set_ylim(-2, 3)
    ax[row,col].grid()
pl.figtext(0.01, 0.5, 'Historical effective radiative forcing (W m$^{-2}$)', rotation=90, va='center')
#ax[0,0].legend(loc='upper left', bbox_to_anchor=[0, -2.4], ncol=3)
ax[0,0].text(0.03, 0.88, 'implied ERF: ensemble members', size=10, transform=ax[0,0].transAxes, color='0.7', backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
ax[0,0].text(0.03, 0.78, 'implied ERF: ensemble mean', size=10, transform=ax[0,0].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
ax[0,0].text(0.03, 0.68, 'Fixed-SST ERF', size=10, transform=ax[0,0].transAxes, color=colors['cmip6'], backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
fig.tight_layout(rect=[0.02, 0, 1, 1])
pl.savefig('../plots/fig2.pdf')

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(30/2.54, 15/2.54))
for i, model in enumerate(models_rfmip):
    col = i%4
    row = i//4
    for run in branch_points['historical'][model].keys():
        if model=='CanESM5' and run=='r1i1p1f1':
            label='implied ERF ensemble members'
        else:
            label=''
        ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model][run], color='0.7', label=label)
    ax[row,col].plot(np.arange(1850.5, 2015), delta_F['historical'][model]['mean'], color='k', label='implied ERF ensemble mean')
    ax[row,col].set_title('(%s) %s' % (chr(97+i), model))
    if '%s TOT' % model in rfmip_erf:
        ax[row,col].plot(np.arange(1850.5, 2015), rfmip_erf['%s TOT' % model].values[:165], color=colors['cmip6'], label='RFMIP ERF')
    ax[row,col].set_xlim(1850, 2015)
    ax[row,col].set_ylim(-2, 3)
    ax[row,col].grid()
pl.figtext(0.01, 0.5, 'Historical effective radiative forcing (W m$^{-2}$)', rotation=90, va='center')
#ax[0,0].legend(loc='upper left', bbox_to_anchor=[0, -2.4], ncol=3)
ax[0,0].text(0.03, 0.88, 'implied ERF: ensemble members', size=10, transform=ax[0,0].transAxes, color='0.7', backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
ax[0,0].text(0.03, 0.78, 'implied ERF: ensemble mean', size=10, transform=ax[0,0].transAxes, backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
ax[0,0].text(0.03, 0.68, 'Fixed-SST ERF', size=10, transform=ax[0,0].transAxes, color=colors['cmip6'], backgroundcolor='w', bbox=dict(boxstyle='square,pad=0.13',ec='w',fc='w'))
fig.tight_layout(rect=[0.02, 0, 1, 1])
pl.savefig('../plots/fig2_agu2021.pdf')

## Aggregate data for plotting fig. 1

In [ ]:
delta_F_cmip6_consolidated = {}
delta_N_cmip6_consolidated = {}
delta_T_cmip6_consolidated = {}
delta_F_cmip6_consolidated_rfmip7 = {}
delta_N_cmip6_consolidated_rfmip7 = {}
delta_T_cmip6_consolidated_rfmip7 = {}
delta_F_rfmip_consolidated = {}

mapping = {
    'historical': 'TOT',
    'hist-GHG': 'GHG',
    'hist-nat': 'NAT',
    'hist-aer': 'AER'
}

for experiment in ['historical', 'hist-nat', 'hist-GHG', 'hist-aer', 'hist-otheranthro']:
    delta_F_cmip6_consolidated[experiment] = np.zeros((165, len(delta_F[experiment])))
    delta_T_cmip6_consolidated[experiment] = np.zeros((165, len(delta_T[experiment])))
    delta_N_cmip6_consolidated[experiment] = np.zeros((165, len(delta_N[experiment])))
    for i, model in enumerate(delta_T[experiment].keys()):
        delta_F_cmip6_consolidated[experiment][:,i] = delta_F[experiment][model]['mean']
        delta_N_cmip6_consolidated[experiment][:,i] = delta_N[experiment][model]['mean']
        delta_T_cmip6_consolidated[experiment][:,i] = delta_T[experiment][model]['mean']
    delta_F_cmip6_consolidated_rfmip7[experiment] = np.zeros((165, 7))
    delta_T_cmip6_consolidated_rfmip7[experiment] = np.zeros((165, 7))
    delta_N_cmip6_consolidated_rfmip7[experiment] = np.zeros((165, 7))
    if experiment!='hist-otheranthro':
        delta_F_rfmip_consolidated[experiment] = np.zeros((165, 7))
    # not GFDL, which didn't do all the DAMIPs.
    for i, model in enumerate(['GISS-E2-1-G', 'MIROC6', 'CNRM-CM6-1', 'CanESM5', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'NorESM2-LM']):
        delta_F_cmip6_consolidated_rfmip7[experiment][:,i] = delta_F[experiment][model]['mean']
        delta_N_cmip6_consolidated_rfmip7[experiment][:,i] = delta_N[experiment][model]['mean']
        delta_T_cmip6_consolidated_rfmip7[experiment][:,i] = delta_T[experiment][model]['mean']
        if experiment!='hist-otheranthro':
            delta_F_rfmip_consolidated[experiment][:,i] = rfmip_erf['%s %s' % (model, mapping[experiment])].values[:165]

In [ ]:
hadcrut5 = pd.read_csv('../data_input/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv', index_col=0)

## Determine CMIP5 forcing and temperature response from Forster et al. (2013)

In [ ]:
delta_N_cmip5 = {}
delta_T_cmip5 = {}
delta_F_cmip5 = {}

models = {}

for experiment in ['historical', 'historicalNat', 'historicalGHG']:
    delta_N_cmip5[experiment] = {}
    delta_T_cmip5[experiment] = {}
    delta_F_cmip5[experiment] = {}
    models[experiment] = set()
    for variable in ['rsdt', 'rsut', 'rlut', 'tas']:
        file_list = (glob.glob('../data_input/cmip5_Forster_etal_2013/%s_*_%s_*.idlsave' % (variable, experiment)))
        for file in file_list:
            models[experiment].add(file.split('_')[5])
    models[experiment] = (list(models[experiment]))
    for model in models[experiment]:
        if model not in lambda5:
            continue
        if model=='CCSM4':
            continue  # it's sick
        delta_N_cmip5[experiment][model] = {}
        delta_T_cmip5[experiment][model] = {}
        delta_F_cmip5[experiment][model] = {}
        file_list = glob.glob('../data_input/cmip5_Forster_etal_2013/rsdt_%s_%s_*.idlsave' % (model, experiment))
        for rsdtfile in file_list:
            run = rsdtfile.split('_')[7][:-8]
            tasfile = rsdtfile.replace('rsdt', 'tas')
            rsutfile = rsdtfile.replace('rsdt', 'rsut')
            rlutfile = rsdtfile.replace('rsdt', 'rlut')
                   
            try:
                tasdata = readsav(tasfile)
                len_tas = len(tasdata['tas'])
                rsdtdata = readsav(rsdtfile)
                len_rsdt = len(rsdtdata['rsdt'])
                rsutdata = readsav(rsutfile)
                len_rsut = len(rsutdata['rsut'])
                rlutdata = readsav(rlutfile)
                len_rlut = len(rlutdata['rlut'])
                tasdata['tas']-tasdata['ctrl'][:len_tas]
                rsdtdata['rsdt']-rsdtdata['ctrl'][:len_tas]
                rsutdata['rsut']-rsutdata['ctrl'][:len_tas]
                rlutdata['rlut']-rlutdata['ctrl'][:len_tas]
            except:
                continue
            
            delta_T_cmip5[experiment][model][run] = np.ones(156) * np.nan
            delta_N_cmip5[experiment][model][run] = np.ones(156) * np.nan
            delta_F_cmip5[experiment][model][run] = np.ones(156) * np.nan
            
            first_time = tasdata['time'][0]
            first_index = int(first_time - 1850)
            if model[:3] == 'Had':
                offset=-11
                first_index = first_index+1
            else:
                offset=0
            print(model, experiment, run, first_time, first_index)
            for i in range(first_index, 156):
                delta_T_cmip5[experiment][model][run][i] = np.mean(tasdata['tas'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(tasdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsdt = np.mean(rsdtdata['rsdt'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsdtdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsut = np.mean(rsutdata['rsut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rlut = np.mean(rlutdata['rlut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rlutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                delta_N_cmip5[experiment][model][run][i] = rsdt - rsut - rlut
            delta_F_cmip5[experiment][model][run] = delta_N_cmip5[experiment][model][run] - lambda5[model] * delta_T_cmip5[experiment][model][run]
        delta_F_array = np.ones((nyears, len(delta_F_cmip5[experiment][model]))) * np.nan
        delta_N_array = np.ones((nyears, len(delta_N_cmip5[experiment][model]))) * np.nan
        delta_T_array = np.ones((nyears, len(delta_T_cmip5[experiment][model]))) * np.nan
        for i, run in enumerate(delta_F_cmip5[experiment][model].keys()):
            delta_F_array[:, i] = delta_F_cmip5[experiment][model][run]
            delta_N_array[:, i] = delta_N_cmip5[experiment][model][run]
            delta_T_array[:, i] = delta_T_cmip5[experiment][model][run]
        delta_F_cmip5[experiment][model]['mean'] = np.nanmean(delta_F_array, axis=1)
        delta_N_cmip5[experiment][model]['mean'] = np.nanmean(delta_N_array, axis=1)
        delta_T_cmip5[experiment][model]['mean'] = np.nanmean(delta_T_array, axis=1)

delta_N_cmip5['historicalOther'] = {}
delta_T_cmip5['historicalOther'] = {}
delta_F_cmip5['historicalOther'] = {}
for model in delta_T_cmip5['historical'].keys():
    if (model not in delta_T_cmip5['historicalNat'].keys()) and (model not in delta_T_cmip5['historicalGHG'].keys()):
        continue
    print(model)
    delta_N_cmip5['historicalOther'][model] = {}
    delta_T_cmip5['historicalOther'][model] = {}
    delta_F_cmip5['historicalOther'][model] = {}
    for run in list(delta_T_cmip5['historical'][model].keys()):
        if (run not in delta_T_cmip5['historicalGHG'][model].keys()) or (run not in delta_T_cmip5['historicalNat'][model].keys()):
            continue
        delta_N_cmip5['historicalOther'][model][run] = delta_N_cmip5['historical'][model][run] - delta_N_cmip5['historicalGHG'][model][run] - delta_N_cmip5['historicalNat'][model][run]
        delta_T_cmip5['historicalOther'][model][run] = delta_T_cmip5['historical'][model][run] - delta_T_cmip5['historicalGHG'][model][run] - delta_T_cmip5['historicalNat'][model][run]
        delta_F_cmip5['historicalOther'][model][run] = delta_F_cmip5['historical'][model][run] - delta_F_cmip5['historicalGHG'][model][run] - delta_F_cmip5['historicalNat'][model][run]
    delta_F_array = np.ones((156, len(delta_F_cmip5['historicalOther'][model]))) * np.nan
    delta_N_array = np.ones((156, len(delta_N_cmip5['historicalOther'][model]))) * np.nan
    delta_T_array = np.ones((156, len(delta_T_cmip5['historicalOther'][model]))) * np.nan
    for i, run in enumerate(delta_T_cmip5['historicalOther'][model].keys()):
        delta_F_array[:, i] = delta_F_cmip5['historicalOther'][model][run]
        delta_N_array[:, i] = delta_N_cmip5['historicalOther'][model][run]
        delta_T_array[:, i] = delta_T_cmip5['historicalOther'][model][run]
    delta_F_cmip5['historicalOther'][model]['mean'] = np.mean(delta_F_array, axis=1)
    delta_N_cmip5['historicalOther'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T_cmip5['historicalOther'][model]['mean'] = np.mean(delta_T_array, axis=1)

In [ ]:
delta_F_cmip5_consolidated = {}
delta_N_cmip5_consolidated = {}
delta_T_cmip5_consolidated = {}
for experiment in ['historical', 'historicalNat', 'historicalGHG', 'historicalOther']:
    delta_F_cmip5_consolidated[experiment] = np.zeros((156, len(delta_F_cmip5[experiment])))
    delta_T_cmip5_consolidated[experiment] = np.zeros((156, len(delta_T_cmip5[experiment])))
    delta_N_cmip5_consolidated[experiment] = np.zeros((156, len(delta_N_cmip5[experiment])))
    for i, model in enumerate(delta_T_cmip5[experiment].keys()):
        delta_F_cmip5_consolidated[experiment][:,i] = delta_F_cmip5[experiment][model]['mean']
        delta_N_cmip5_consolidated[experiment][:,i] = delta_N_cmip5[experiment][model]['mean']
        delta_T_cmip5_consolidated[experiment][:,i] = delta_T_cmip5[experiment][model]['mean']

In [ ]:
nmodels = {}
nmodels['CMIP5'] = {}
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    count = 0
    for row in delta_F_cmip5_consolidated[expt].T:
        count = count + (1-np.all(np.isnan(row)))
    nmodels['CMIP5'][expt]=count
nmodels['CMIP6'] = {}
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    count = 0
    for row in delta_F_cmip6_consolidated[expt].T:
        count = count + (1-np.all(np.isnan(row)))
    nmodels['CMIP6'][expt]=count

## How many models?

In [ ]:
nmodels

In [ ]:
goodmodels = {}
sens = {}
goodmodels['CMIP5'] = {}
sens['CMIP5'] = {}
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    goodlist = []
    senslist = []
    for model in delta_F_cmip5[expt]:
        if not (np.all(np.isnan(delta_F_cmip5[expt][model]['mean']))):
            goodlist.append(model)
            senslist.append(-1 / lambda5[model])
    goodmodels['CMIP5'][expt]=goodlist
    sens['CMIP5'][expt] = np.array(senslist)
goodmodels['CMIP6'] = {}
sens['CMIP6'] = {}
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    goodlist = []
    senslist = []
    for model in delta_F[expt]:
        if not (np.all(np.isnan(delta_F[expt][model]['mean']))):
            goodlist.append(model)
            senslist.append(-1 / lambda6[model])
    goodmodels['CMIP6'][expt]=goodlist
    sens['CMIP6'][expt] = np.array(senslist)

## Data for table 1

In [ ]:
for expt in ['historical','hist-GHG','hist-nat','hist-aer','hist-otheranthro']:
    print(expt, sens['CMIP6'][expt].mean(), sens['CMIP6'][expt].std())
for expt in ['historical','historicalGHG','historicalNat','historicalOther']:
    print(expt, sens['CMIP5'][expt].mean(), sens['CMIP5'][expt].std())

## Plot fig. 1

In [ ]:
fig, ax = pl.subplots(4,2,figsize=(19/2.54, 23/2.54))

cmip5_names = {
    'historical': 'historical',
    'hist-GHG': 'historicalGHG',
    'hist-nat': 'historicalNat',
    'hist-otheranthro': 'historicalOther',
}

legend_pos = {
    'historical': 'upper left',
    'hist-GHG': 'upper left',
    'hist-nat': 'lower center',
    'hist-otheranthro': 'lower left'
}

for i, expt in enumerate(['historical', 'hist-GHG','hist-nat','hist-otheranthro']):
    ax[i,1].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_F_cmip6_consolidated[expt], axis=1)-np.std(delta_F_cmip6_consolidated[expt], axis=1),
        np.mean(delta_F_cmip6_consolidated[expt], axis=1)+np.std(delta_F_cmip6_consolidated[expt], axis=1),
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2015),
        np.mean(delta_F_cmip6_consolidated[expt], axis=1),
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color=colors['cmip6']
    )
    ax[i,1].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5',
        color=colors['cmip5']
    )
    if i==3:
        ax[3,1].plot(np.arange(1850.5,2015), np.mean(delta_F_cmip6_consolidated['hist-aer'], axis=1), color='darkblue', lw=1.0)
    ax[i,1].set_ylabel('W m$^{-2}$')
    ax[i,1].grid()
    ax[i,1].set_title(expt)
    ax[i,1].set_xlim(1850, 2015)
    #ax[row,col].set_ylim(-2, 3)

    ax[i,0].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_T_cmip6_consolidated[expt], axis=1)-np.std(delta_T_cmip6_consolidated[expt], axis=1),
        np.mean(delta_T_cmip6_consolidated[expt], axis=1)+np.std(delta_T_cmip6_consolidated[expt], axis=1),
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2015),
        np.mean(delta_T_cmip6_consolidated[expt], axis=1),
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color=colors['cmip6']
    )
    ax[i,0].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5 (%d)' % (nmodels['CMIP5'][cmip5_names[expt]]),
        color=colors['cmip5']
    )
    if i==0:
        ax[0,0].plot(
            hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
            color='k', 
            label='HadCRUT5',
            lw=1.0
        )
    if i==3:
        ax[3,0].plot(
            np.arange(1850.5,2015),
            np.mean(delta_T_cmip6_consolidated['hist-aer'], axis=1),
            label='CMIP6 aerosol-only (%d)' % (nmodels['CMIP6']['hist-aer']),
            color='darkblue',
            lw=1.0,
        )

    ax[i,0].set_ylabel('$^{\circ}$C')
    ax[i,0].grid()
    ax[i,0].set_title(expt)
    ax[i,0].set_xlim(1850, 2015)
    ax[i,0].legend(loc=legend_pos[expt])

ax[0,1].set_ylim(-2,2.5)
ax[0,0].set_ylim(-0.5,1.4)
ax[1,1].set_ylim(-0.4,3.2)
ax[1,0].set_ylim(-0.2,2)
ax[2,1].set_ylim(-3,1)
ax[2,0].set_ylim(-0.6,0.25)
ax[3,1].set_ylim(-2,0.7)
ax[3,0].set_ylim(-1.05,0.2)

ax[0,1].set_title('(b) Implied total ERF ($\Delta F$)')
ax[1,1].set_title('(d) Implied greenhouse gas ERF ($\Delta F_{\mathrm{GHG}}$)')
ax[2,1].set_title('(f) Implied natural ERF ($\Delta F_{\mathrm{nat}}$)')
ax[3,1].set_title('(h) Implied other anthropogenic ERF ($\Delta F_{\mathrm{other}}$)')
ax[0,0].set_title('(a) Total warming ($\Delta T$)')
ax[1,0].set_title('(c) Greenhouse gas warming ($\Delta T_{\mathrm{GHG}}$)')
ax[2,0].set_title('(e) Natural warming ($\Delta T_{\mathrm{nat}}$)')
ax[3,0].set_title('(g) Other anthropogenic warming ($\Delta T_{\mathrm{other}}$)')

ax00 = ax[0,0].axis()
rec = Rectangle((ax00[0]-33,ax00[2]-0.4),(ax00[1]-ax00[0])+41,(ax00[3]-ax00[2])+0.8,fill=False,lw=1,ls='--')
rec = ax[0,0].add_patch(rec)
rec.set_clip_on(False)

fig.tight_layout()
pl.savefig('../plots/fig1.pdf')
pl.savefig('../plots/fig1.png')  # for GitHub

## Reviewer response showing just the 7 RFMIP models + RFMIP forcing

In [ ]:
fig, ax = pl.subplots(4,2,figsize=(19/2.54, 23/2.54))

cmip5_names = {
    'historical': 'historical',
    'hist-GHG': 'historicalGHG',
    'hist-nat': 'historicalNat',
    'hist-otheranthro': 'historicalOther',
}

legend_pos = {
    'historical': 'upper left',
    'hist-GHG': 'upper left',
    'hist-nat': 'lower center',
    'hist-otheranthro': 'lower left'
}

for i, expt in enumerate(['historical', 'hist-GHG','hist-nat','hist-otheranthro']):
    ax[i,1].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_F_cmip6_consolidated_rfmip7[expt], axis=1)-np.std(delta_F_cmip6_consolidated_rfmip7[expt], axis=1),
        np.mean(delta_F_cmip6_consolidated_rfmip7[expt], axis=1)+np.std(delta_F_cmip6_consolidated_rfmip7[expt], axis=1),
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2015),
        np.mean(delta_F_cmip6_consolidated_rfmip7[expt], axis=1),
        label='CMIP6 (7)',
        color=colors['cmip6']
    )
    if expt!='hist-otheranthro':
        ax[i,1].fill_between(
            np.arange(1850.5,2015), 
            np.mean(delta_F_rfmip_consolidated[expt], axis=1)-np.std(delta_F_rfmip_consolidated[expt], axis=1),
            np.mean(delta_F_rfmip_consolidated[expt], axis=1)+np.std(delta_F_rfmip_consolidated[expt], axis=1),
            color='k',
            alpha=0.2,
            edgecolor=None
        )
        ax[i,1].plot(
            np.arange(1850.5,2015),
            np.mean(delta_F_rfmip_consolidated[expt], axis=1),
            label='RFMIP (7)',
            color='k'
        )
    ax[i,1].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_F_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5',
        color=colors['cmip5']
    )
    if i==3:
        ax[3,1].plot(np.arange(1850.5,2015), np.mean(delta_F_cmip6_consolidated_rfmip7['hist-aer'], axis=1), color='darkblue', lw=1.0)
        ax[3,1].fill_between(
            np.arange(1850.5,2015), 
            np.mean(delta_F_rfmip_consolidated['hist-aer'], axis=1)-np.std(delta_F_rfmip_consolidated['hist-aer'], axis=1),
            np.mean(delta_F_rfmip_consolidated['hist-aer'], axis=1)+np.std(delta_F_rfmip_consolidated['hist-aer'], axis=1),
            color='k',
            alpha=0.2,
            edgecolor=None
        )
        ax[3,1].plot(
            np.arange(1850.5,2015),
            np.mean(delta_F_rfmip_consolidated['hist-aer'], axis=1),
            label='RFMIP (7)',
            color='k'
        )
    ax[i,1].set_ylabel('W m$^{-2}$')
    ax[i,1].grid()
    ax[i,1].set_title(expt)
    ax[i,1].set_xlim(1850, 2015)
    #ax[row,col].set_ylim(-2, 3)

    ax[i,0].fill_between(
        np.arange(1850.5,2015), 
        np.mean(delta_T_cmip6_consolidated_rfmip7[expt], axis=1)-np.std(delta_T_cmip6_consolidated_rfmip7[expt], axis=1),
        np.mean(delta_T_cmip6_consolidated_rfmip7[expt], axis=1)+np.std(delta_T_cmip6_consolidated_rfmip7[expt], axis=1),
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2015),
        np.mean(delta_T_cmip6_consolidated_rfmip7[expt], axis=1),
        label='CMIP6 (7)',
        color=colors['cmip6']
    )
    ax[i,0].fill_between(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2006),
        np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
        label='CMIP5 (%d)' % (nmodels['CMIP5'][cmip5_names[expt]]),
        color=colors['cmip5']
    )
    if i==0:
        ax[0,0].plot(
            hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
            color='k', 
            label='HadCRUT5',
            lw=1.0
        )
    if i==3:
        ax[3,0].plot(
            np.arange(1850.5,2015),
            np.mean(delta_T_cmip6_consolidated_rfmip7['hist-aer'], axis=1),
            label='CMIP6 aerosol-only (%d)' % (nmodels['CMIP6']['hist-aer']),
            color='darkblue',
            lw=1.0,
        )

    ax[i,0].set_ylabel('$^{\circ}$C')
    ax[i,0].grid()
    ax[i,0].set_title(expt)
    ax[i,0].set_xlim(1850, 2015)
    ax[i,0].legend(loc=legend_pos[expt])

ax[0,1].set_ylim(-2,2.5)
ax[0,0].set_ylim(-0.5,1.8)
ax[1,1].set_ylim(-1,3.2)
ax[1,0].set_ylim(-0.2,2.3)
ax[2,1].set_ylim(-3,1)
ax[2,0].set_ylim(-0.6,0.25)
ax[3,1].set_ylim(-2,1)
ax[3,0].set_ylim(-1.15,0.3)

ax[0,1].set_title('(b) Implied total ERF ($\Delta F$)')
ax[1,1].set_title('(d) Implied greenhouse gas ERF ($\Delta F_{\mathrm{GHG}}$)')
ax[2,1].set_title('(f) Implied natural ERF ($\Delta F_{\mathrm{nat}}$)')
ax[3,1].set_title('(h) Implied other anthropogenic ERF ($\Delta F_{\mathrm{other}}$)')
ax[0,0].set_title('(a) Total warming ($\Delta T$)')
ax[1,0].set_title('(c) Greenhouse gas warming ($\Delta T_{\mathrm{GHG}}$)')
ax[2,0].set_title('(e) Natural warming ($\Delta T_{\mathrm{nat}}$)')
ax[3,0].set_title('(g) Other anthropogenic warming ($\Delta T_{\mathrm{other}}$)')

ax00 = ax[0,0].axis()
rec = Rectangle((ax00[0]-33,ax00[2]-0.4),(ax00[1]-ax00[0])+41,(ax00[3]-ax00[2])+0.8,fill=False,lw=1,ls='--')
rec = ax[0,0].add_patch(rec)
rec.set_clip_on(False)

fig.tight_layout()
pl.savefig('../plots/fig1_rfmip7.png')

## Prepare CanESM5 ensemble for fig. 3

In [ ]:
delta_F_canesm5_consolidated = {}
delta_T_canesm5_consolidated = {}

for experiment in ['historical', 'hist-nat', 'hist-GHG', 'hist-otheranthro']:
    delta_F_canesm5_consolidated[experiment] = {}
    delta_T_canesm5_consolidated[experiment] = {}
    delta_F_canesm5_consolidated[experiment + '-cmip5'] = {}
    delta_T_canesm5_consolidated[experiment + '-cmip5'] = {}
    delta_F_canesm5_consolidated[experiment]['mean'] = np.mean(
        (
            delta_F[experiment]['CanESM5']['r1i1p1f1'],
            delta_F[experiment]['CanESM5']['r2i1p1f1'],
            delta_F[experiment]['CanESM5']['r3i1p1f1'],
            delta_F[experiment]['CanESM5']['r4i1p1f1'],
            delta_F[experiment]['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_T_canesm5_consolidated[experiment]['mean'] = np.mean(
        (
            delta_T[experiment]['CanESM5']['r1i1p1f1'],
            delta_T[experiment]['CanESM5']['r2i1p1f1'],
            delta_T[experiment]['CanESM5']['r3i1p1f1'],
            delta_T[experiment]['CanESM5']['r4i1p1f1'],
            delta_T[experiment]['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_F_canesm5_consolidated[experiment]['std'] = np.std(
        (
            delta_F[experiment]['CanESM5']['r1i1p1f1'],
            delta_F[experiment]['CanESM5']['r2i1p1f1'],
            delta_F[experiment]['CanESM5']['r3i1p1f1'],
            delta_F[experiment]['CanESM5']['r4i1p1f1'],
            delta_F[experiment]['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_T_canesm5_consolidated[experiment]['std'] = np.std(
        (
            delta_T[experiment]['CanESM5']['r1i1p1f1'],
            delta_T[experiment]['CanESM5']['r2i1p1f1'],
            delta_T[experiment]['CanESM5']['r3i1p1f1'],
            delta_T[experiment]['CanESM5']['r4i1p1f1'],
            delta_T[experiment]['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_F_canesm5_consolidated[experiment + '-cmip5']['mean'] = np.mean(
        (
            delta_F[experiment + '-cmip5']['CanESM5']['r1i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r2i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r3i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r4i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_T_canesm5_consolidated[experiment + '-cmip5']['mean'] = np.mean(
        (
            delta_T[experiment + '-cmip5']['CanESM5']['r1i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r2i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r3i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r4i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_F_canesm5_consolidated[experiment + '-cmip5']['std'] = np.std(
        (
            delta_F[experiment + '-cmip5']['CanESM5']['r1i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r2i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r3i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r4i1p1f1'],
            delta_F[experiment + '-cmip5']['CanESM5']['r5i1p1f1'],
        ), axis=0
    )
    delta_T_canesm5_consolidated[experiment + '-cmip5']['std'] = np.std(
        (
            delta_T[experiment + '-cmip5']['CanESM5']['r1i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r2i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r3i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r4i1p1f1'],
            delta_T[experiment + '-cmip5']['CanESM5']['r5i1p1f1'],
        ), axis=0
    )

## Plot fig. 3

In [ ]:
fig, ax = pl.subplots(4,2,figsize=(19/2.54, 23/2.54))

for i, expt in enumerate(['historical', 'hist-GHG','hist-nat','hist-otheranthro']):
    ax[i,1].fill_between(
        np.arange(1850.5,2015), 
        delta_F_canesm5_consolidated[expt]['mean']-delta_F_canesm5_consolidated[expt]['std'],
        delta_F_canesm5_consolidated[expt]['mean']+delta_F_canesm5_consolidated[expt]['std'],
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2015),
        delta_F_canesm5_consolidated[expt]['mean'],
        label='CMIP6 (%d)' % (nmodels['CMIP6'][expt]),
        color=colors['cmip6']
    )
    ax[i,1].fill_between(
        np.arange(1850.5,2006), 
        delta_F_canesm5_consolidated[expt + '-cmip5']['mean']-delta_F_canesm5_consolidated[expt + '-cmip5']['std'],
        delta_F_canesm5_consolidated[expt + '-cmip5']['mean']+delta_F_canesm5_consolidated[expt + '-cmip5']['std'],
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,1].plot(
        np.arange(1850.5,2006),
        delta_F_canesm5_consolidated[expt + '-cmip5']['mean'],
        label='CMIP5',
        color=colors['cmip5']
    )
    ax[i,1].set_ylabel('W m$^{-2}$')
    ax[i,1].grid()
    ax[i,1].set_title(expt)
    ax[i,1].set_xlim(1850, 2015)
    #ax[row,col].set_ylim(-2, 3)

    if i==0:
        label = 'CanESM5, CMIP6 forcing'
    else:
        label = ''
        
    ax[i,0].fill_between(
        np.arange(1850.5,2015), 
        delta_T_canesm5_consolidated[expt]['mean']-delta_T_canesm5_consolidated[expt]['std'],
        delta_T_canesm5_consolidated[expt]['mean']+delta_T_canesm5_consolidated[expt]['std'],
        color=colors['cmip6'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2015),
        delta_T_canesm5_consolidated[expt]['mean'],
        label=label,
        color=colors['cmip6']
    )
    if i==0:
        label = 'CanESM5, CMIP5 forcing'
    else:
        label = ''
    ax[i,0].fill_between(
        np.arange(1850.5,2006), 
        delta_T_canesm5_consolidated[expt + '-cmip5']['mean']-delta_T_canesm5_consolidated[expt + '-cmip5']['std'],
        delta_T_canesm5_consolidated[expt + '-cmip5']['mean']+delta_T_canesm5_consolidated[expt + '-cmip5']['std'],
        color=colors['cmip5'],
        alpha=0.2,
        edgecolor=None
    )
    ax[i,0].plot(
        np.arange(1850.5,2006),
        delta_T_canesm5_consolidated[expt + '-cmip5']['mean'],
        label=label,
        color=colors['cmip5']
    )
    if i==0:
        ax[0,0].plot(
            hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
            color='k', 
            label='HadCRUT5',
            lw=1.0
        )
        ax[i,0].legend()

    ax[i,0].set_ylabel('$^{\circ}$C')
    ax[i,0].grid()
    ax[i,0].set_title(expt)
    ax[i,0].set_xlim(1850, 2015)


ax[0,1].set_ylim(-2,2.5)
ax[0,0].set_ylim(-0.6,1.9)
ax[1,1].set_ylim(-0.7,3.2)
ax[1,0].set_ylim(-0.4,2.5)
ax[2,1].set_ylim(-2.4,1)
ax[2,0].set_ylim(-0.7,0.4)
ax[3,1].set_ylim(-2,1)
ax[3,0].set_ylim(-1.25,0.4)

ax[0,1].set_title('(b) Implied total ERF ($\Delta F$)')
ax[1,1].set_title('(d) Implied greenhouse gas ERF ($\Delta F_{\mathrm{GHG}}$)')
ax[2,1].set_title('(f) Implied natural ERF ($\Delta F_{\mathrm{nat}}$)')
ax[3,1].set_title('(h) Implied aerosol ERF ($\Delta F_{\mathrm{aer}}$)')
ax[0,0].set_title('(a) Total warming ($\Delta T$)')
ax[1,0].set_title('(c) Greenhouse gas warming ($\Delta T_{\mathrm{GHG}}$)')
ax[2,0].set_title('(e) Natural warming ($\Delta T_{\mathrm{nat}}$)')
ax[3,0].set_title('(g) Aerosol warming ($\Delta T_{\mathrm{aer}}$)')
fig.tight_layout()
pl.savefig('../plots/fig3.pdf')

## Data for Table 2

In [ ]:
for expt in ['historical', 'hist-GHG','hist-nat','hist-otheranthro','hist-aer']:
    print('%16s & $%.2f \pm %.2f$ & $%.2f \pm %.2f$ & $%.2f$ &' % 
        (
            expt,
            np.mean(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.std(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.mean(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.std(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)),
            np.mean(np.mean(delta_T_cmip6_consolidated[expt][145:156,:], axis=0)) / np.mean(np.mean(delta_F_cmip6_consolidated[expt][145:156,:], axis=0))
        )
    )

In [ ]:
for expt in ['historical', 'historicalGHG','historicalNat','historicalOther']:
    print('$%.2f \pm %.2f$ & $%.2f \pm %.2f$ & $%.2f$ \\\\' % 
        (
            np.nanmean(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanstd(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanmean(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanstd(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)),
            np.nanmean(np.nanmean(delta_T_cmip5_consolidated[expt][145:156,:], axis=0)) / np.nanmean(np.nanmean(delta_F_cmip5_consolidated[expt][145:156,:], axis=0))
        )
    )

# Plot fig. S3

Reviewer 1 asks if it is the case that higher sensitivity models have weaker GHG forcing and stronger aerosol.

In [ ]:
damip_cmip6 = [
    'ACCESS-CM2',
    'ACCESS-ESM1-5',
    'CanESM5',
    'CESM2',
    'CNRM-CM6-1',
    'FGOALS-g3',
    'GFDL-ESM4',
    'GISS-E2-1-G',
    'HadGEM3-GC31-LL',
    'IPSL-CM6A-LR',
    'MIROC6',
    'MRI-ESM2-0',
    'NorESM2-LM'
]

damip_cmip5 = [
    'bcc-csm1-1',
    'CanESM2',
    'CNRM-CM5',
    'CSIRO-Mk3-6-0',
    'GFDL-CM3',
    'GFDL-ESM2M',
    'GISS-E2-H',
    'GISS-E2-R',
    'HadGEM2-ES',
    'IPSL-CM5A-LR',
    'MIROC-ESM',
    'MRI-CGCM3',
    'NorESM1-M'
]

hist_clim_sens6 = np.zeros(13)
ghg_forcing6 = np.zeros(13)
aer_forcing6 = np.zeros(13)

hist_clim_sens5 = np.zeros(13)
ghg_forcing5 = np.zeros(13)
aer_forcing5 = np.zeros(13)

fig, ax=pl.subplots(1,2,figsize=(19.0/2.54, 10.0/2.54))
for i, model in enumerate(damip_cmip6):
    hist_clim_sens6[i] = delta_T['historical'][model]['mean'][145:156].mean() / delta_F['historical'][model]['mean'][145:156].mean()
    ghg_forcing6[i] = delta_F['hist-GHG'][model]['mean'][145:156].mean()
    aer_forcing6[i] = delta_F['hist-otheranthro'][model]['mean'][145:156].mean()
for i, model in enumerate(damip_cmip5):
    hist_clim_sens5[i] = delta_T_cmip5['historical'][model]['mean'][145:156].mean() / delta_F_cmip5['historical'][model]['mean'][145:156].mean()
    ghg_forcing5[i] = delta_F_cmip5['historicalGHG'][model]['mean'][145:156].mean()
    aer_forcing5[i] = delta_F_cmip5['historicalOther'][model]['mean'][145:156].mean()
ax[0].scatter(hist_clim_sens6, ghg_forcing6, color=colors['cmip6'], label='CMIP6')
ax[1].scatter(hist_clim_sens6, aer_forcing6, color=colors['cmip6'])
ax[0].scatter(hist_clim_sens5, ghg_forcing5, color=colors['cmip5'], label='CMIP5')
ax[1].scatter(hist_clim_sens5, aer_forcing5, color=colors['cmip5'])
regress_ghg6 = linregress(hist_clim_sens6, ghg_forcing6)
regress_aer6 = linregress(hist_clim_sens6, aer_forcing6)
regress_ghg5 = linregress(hist_clim_sens5, ghg_forcing5)
regress_aer5 = linregress(hist_clim_sens5, aer_forcing5)
ax[0].plot(np.linspace(0.35,0.8), np.linspace(0.35,0.8)*regress_ghg6.slope + regress_ghg6.intercept, color=colors['cmip6'])
ax[1].plot(np.linspace(0.35,0.8), np.linspace(0.35,0.8)*regress_aer6.slope + regress_aer6.intercept, color=colors['cmip6'])
ax[0].plot(np.linspace(0.25,0.65), np.linspace(0.25,0.65)*regress_ghg5.slope + regress_ghg5.intercept, color=colors['cmip5'])
ax[1].plot(np.linspace(0.25,0.65), np.linspace(0.25,0.65)*regress_aer5.slope + regress_aer5.intercept, color=colors['cmip5'])
print(regress_ghg6.pvalue, regress_aer6.pvalue)
print(regress_ghg5.pvalue, regress_aer5.pvalue)
ax[0].legend()


ax[0].set_xlabel('Historical climate sensitivity, K W$^{-1}$ m$^2$')
ax[1].set_xlabel('Historical climate sensitivity, K W$^{-1}$ m$^2$')
ax[0].set_ylabel('Greenhouse gas implied ERF, W m$^{-2}$')
ax[1].set_ylabel('Other anthropogenic implied ERF, W m$^{-2}$')

ax[0].set_xlim(0.25, 0.8)
ax[0].set_ylim(1.45, 3.3)
ax[1].set_xlim(0.25, 0.8)
ax[1].set_ylim(-1.75, 0.2)

ax[0].set_title('(a) Greenhouse gas implied ERF')
ax[1].set_title('(b) Other anthropogenic implied ERF')

fig.tight_layout()
pl.savefig('../plots/figS3.pdf')

# Plot alternative fig. 1a based on internal variability as the uncertainty

Was a response to reviewer 2

In [ ]:
fig, ax = pl.subplots(figsize=(12/2.54, 9.5/2.54))

expt = 'historical'

ax.fill_between(
    np.arange(1850.5,2015), 
#    np.mean(delta_T_cmip6_consolidated[expt], axis=1)-np.std(delta_T_cmip6_consolidated[expt], axis=1),
#    np.mean(delta_T_cmip6_consolidated[expt], axis=1)+np.std(delta_T_cmip6_consolidated[expt], axis=1),
    np.mean(delta_T_cmip6_consolidated[expt], axis=1)-0.11,
    np.mean(delta_T_cmip6_consolidated[expt], axis=1)+0.11,
    color=colors['cmip6'],
    alpha=0.2,
    edgecolor=None
)
ax.plot(
    np.arange(1850.5,2015),
    np.mean(delta_T_cmip6_consolidated[expt], axis=1),
    label='CMIP6 (%d models)' % (nmodels['CMIP6'][expt]),
    color=colors['cmip6']
)
ax.fill_between(
    np.arange(1850.5,2006),
    #np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)-np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
    #np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)+np.nanstd(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
    np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)-0.11,
    np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1)+0.11,
    color=colors['cmip5'],
    alpha=0.2,
    edgecolor=None
)
ax.plot(
    np.arange(1850.5,2006),
    np.nanmean(delta_T_cmip5_consolidated[cmip5_names[expt]], axis=1),
    label='CMIP5 (%d models)' % (nmodels['CMIP5'][cmip5_names[expt]]),
    color=colors['cmip5']
)

ax.plot(
    hadcrut5.loc[:2020, 'Anomaly (deg C)'] - hadcrut5.loc[1850:1900,'Anomaly (deg C)'].mean(),
    color='k', 
    label='HadCRUT5',
    lw=1.0
)

ax.set_ylabel('$^{\circ}$C')
ax.grid()
ax.set_xlim(1850, 2015)
ax.legend()
ax.set_ylim(-0.4,1.4)

ax.set_title('Total historical warming ($\Delta T$)')

fig.tight_layout()
pl.savefig('../plots/fig1a_intvar_uncertainty.png')

## Get list of CMIP5 models and variants for table S1

In [ ]:
for expt in ['historical', 'historicalGHG', 'historicalNat']:
    print(expt)
    print('----------')
    for model in delta_T_cmip5[expt].keys():
        for run in delta_T_cmip5[expt][model].keys():
            if run=='mean':
                continue
            print(model, run, np.sum(~np.isnan(delta_T_cmip5[expt][model][run])))
        print()
    print()

## What is warming expressed relative to 1850-1900?

A positive anomaly for 1850-1900 would make "present day" warming less using this baseline period. Hence, the gap between CMIP5 and CMIP6 would be even wider if we had used 1850-1900 rather than model-derived pre-industrial controls.

In [ ]:
delta_T_cmip6_consolidated['historical'][:51].mean()

In [ ]:
np.nanmean(delta_T_cmip5_consolidated['historical'][:51])